In [0]:
# IMPORTS
import requests
import json
import time
import os
import pandas as pd
from datetime import datetime

In [0]:
# RAW NOTEBOOK - con función para el orquestador

def run_raw(audit_date: str) -> str:
    """
    Ejecuta el scraping de Mercadona para una fecha dada
    y guarda los datos en Delta Lake particionados por audit_date.
    Devuelve "OK" si todo salió bien.
    """

    print(f"=== INICIO RAW - audit_date={audit_date} ===")

    # Carpeta en DBFS para JSONs
    dbfs_folder = "/dbfs/FileStore/json_categorias"
    os.makedirs(dbfs_folder, exist_ok=True)

    # Cabeceras para requests
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/117.0.0.0 Safari/537.36"
    }

    # IDs de categorías a scrapear
    all_category_ids = [112, 115, 156, 118, 77, 48,
                        86, 99, 897, 226, 78, 159,
                        135, 164, 168, 169]

    # --- SCRAPING ---
    print("=== INICIO SCRAPING ===")
    for cat_id in all_category_ids:
        url = f"https://tienda.mercadona.es/api/categories/{cat_id}/"
        try:
            r = requests.get(url, headers=headers)
            if r.status_code == 200:
                productos = r.json()
                filename = f"{dbfs_folder}/categoria_{cat_id}_{audit_date}.json"
                with open(filename, "w", encoding="utf-8") as f:
                    json.dump(productos, f, ensure_ascii=False, indent=4)
                print(f"[OK] Guardado JSON categoría {cat_id} para {audit_date}")
            else:
                print(f"[ERROR] {r.status_code} en categoría {cat_id}")
        except Exception as e:
            print(f"[ERROR] en categoría {cat_id}: {e}")
        time.sleep(0.5)
    print("=== FIN SCRAPING ===")

    # --- TRANSFORMACIÓN ---
    print("=== INICIO TRANSFORMACIÓN ===")
    productos_lista = []
    for cat_id in all_category_ids:
        file_path = f"/dbfs/FileStore/json_categorias/categoria_{cat_id}_{audit_date}.json"
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            for cat in data.get("categories", []):
                if "products" in cat:
                    primeros_6 = cat["products"][:6]
                    for p in primeros_6:
                        price = p.get("price_instructions", {})
                        productos_lista.append({
                            "id_producto": p.get("id", ""),
                            "nombre": p.get("display_name", ""),
                            "categoria": cat.get("name", ""),
                            "categoria_id": cat_id,
                            "packaging": p.get("packaging", ""),
                            "precio_unitario": price.get("unit_price", None),
                            "precio_pack": price.get("bulk_price", None),
                            "precio_referencia": price.get("reference_price", None),
                            "formato_referencia": price.get("reference_format", ""),
                            "precio_anterior": price.get("previous_unit_price", None),
                            "iva": price.get("iva", None),
                            "precio_bajado": price.get("price_decreased", False),
                            "tamano_unidad": price.get("unit_size", None),
                            "nombre_unidad": price.get("unit_name", ""),
                            "unidades_totales": price.get("total_units", None),
                            "fecha_no_disponible": p.get("unavailable_from", None),
                            "audit_date": audit_date
                        })
        except FileNotFoundError:
            print(f"[WARN] No se encontró {file_path}")

    df = pd.DataFrame(productos_lista)
    print(f"[OK] Transformación completada con {len(df)} productos")

    # --- CARGA EN DELTA ---
    print("=== INICIO CARGA EN DELTA LAKE ===")
    df_spark = spark.createDataFrame(df)
    delta_path_partitioned = "/mnt/datalake/tfm/mercadona/raw/"

    df_spark.write.format("delta") \
        .mode("append") \
        .partitionBy("audit_date") \
        .save(delta_path_partitioned)

    print(f"[OK] Datos cargados en Delta Lake particionados por audit_date={audit_date}")
    print("=== FIN RAW ===")

    return "OK"


# --- Entrada desde orquestador ---
dbutils.widgets.text("run_date", "")
audit_date = dbutils.widgets.get("run_date")

result = run_raw(audit_date)
dbutils.notebook.exit(result)